## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="https://github.com/bmauss/dsc-resilient-distributed-datasets-rdd-lab-onl01-dtsc-ft-052620/blob/master/images/rdd_diagram.png?raw=1" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [5]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 60kB/s 
     |████████████████████████████████| 204kB 35.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257928 sha256=787bf981e1887f7f39be649f5150adb8fd7a241156062933764e0c66e55f77fa
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [6]:
import pyspark
pyspark.__version__

'2.4.5'

In [7]:
import os
# /usr/lib/jvm/java-8-openjdk-amd64
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# /content/spark-2.4.5-bin-hadoop2.7
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [8]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [10]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [13]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [14]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [15]:
# count
rdd.count()

1000

In [16]:
# first
rdd.first()

1

In [19]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [21]:
# top
rdd.top(6)

[1000, 999, 998, 997, 996, 995]

In [22]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [23]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([4.27523811e-01, 4.03151960e-01, 2.27194835e+00, 1.18918404e+00,
       4.80593445e+00, 4.39463052e+00, 2.60679735e+00, 1.54323476e+00,
       3.31137957e+00, 8.61547237e+00, 8.64744818e+00, 8.16464149e+00,
       2.76083796e+00, 1.13233987e+01, 9.18034299e+00, 6.95053885e+00,
       1.09385675e+01, 5.30052445e+00, 3.69006176e+00, 1.05298487e+01,
       5.81567117e+00, 1.74795162e+01, 5.75522328e+00, 8.13852143e-02,
       6.09322446e+00, 1.77283423e+01, 8.60474088e+00, 8.78797092e+00,
       1.17422047e+01, 1.41777120e+01, 5.21996889e+00, 1.12454185e+01,
       1.65785219e+01, 1.54208226e+01, 4.90701592e-01, 1.44817147e+01,
       1.65351696e+01, 6.92161657e+00, 2.96695921e+01, 4.76598422e+00,
       5.92928571e+00, 2.89410904e+01, 1.44324663e+01, 3.74865136e+01,
       4.44407416e+00, 4.50528750e+01, 2.37996432e+01, 6.98525392e+00,
       1.10554949e+01, 3.23196363e+01, 2.66573172e+01, 2.67036338e+01,
       3.47813006e+01, 2.38815072e+01, 1.84079327e+00, 4.01855015e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [24]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(15)

[0.4275238112926609,
 0.4031519601395639,
 2.2719483477500684,
 1.189184042641433,
 4.805934446219367,
 4.394630515020783,
 2.6067973520865957,
 1.5432347580948118,
 3.3113795659948613,
 8.615472365367715,
 8.647448178226801,
 8.164641486744665,
 2.760837961125134,
 11.32339869193781,
 9.18034298741222]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [25]:
def sales_tax(num):
    # deducting 0.8 since sales tax will be deducted from the revenue
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [29]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.3933219063892481,
 0.3708998033283988,
 2.090192479930063,
 1.0940493192301184,
 4.421459690521818,
 4.04306007381912,
 2.3982535639196683,
 1.419775977447227,
 3.0464692007152725,
 7.926234576138298]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [30]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [31]:
discounted.take(10)

[0.3539897157503233,
 0.3338098229955589,
 1.8811732319370567,
 0.9846443873071066,
 3.9793137214696364,
 3.6387540664372082,
 2.1584282075277015,
 1.2777983797025043,
 2.741822280643745,
 7.133611118524469]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [32]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[790.8562001839473,
 778.416011876469,
 758.9034230884556,
 744.2871649451679,
 740.910261804524,
 723.3279735391712,
 716.8276809778097,
 715.1981084493444,
 707.6745942372395,
 704.0601837899915,
 702.3661735218192,
 691.1778701066436,
 669.3819944617372,
 665.3337428856335,
 655.5376494257858]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [33]:
discounted.toDebugString()

b'(10) PythonRDD[13] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[6] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [34]:
mapped = price_items.map( lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.4275238112926609, 0.3539897157503233), (0.4031519601395639, 0.3338098229955589), (2.2719483477500684, 1.8811732319370567), (1.189184042641433, 0.9846443873071066), (4.805934446219367, 3.9793137214696364), (4.394630515020783, 3.6387540664372082), (2.6067973520865957, 2.1584282075277015), (1.5432347580948118, 1.2777983797025043), (3.3113795659948613, 2.741822280643745), (8.615472365367715, 7.133611118524469)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [35]:
flat_mapped = price_items.flatMap(lambda x: (x, x*.92 *.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.4275238112926609, 0.3539897157503233, 0.4031519601395639, 0.3338098229955589, 2.2719483477500684, 1.8811732319370567, 1.189184042641433, 0.9846443873071066, 4.805934446219367, 3.9793137214696364]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [36]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

263

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [37]:
selected_items.reduce(lambda x, y: x + y)

119704.56986320377

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [38]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(17, 308.83696880679133),
 (15, 307.4221871227305),
 (17, 334.40267937557934),
 (15, 320.4974278129902),
 (48, 331.77261352708535),
 (47, 356.2977413803764),
 (35, 302.62373477816294)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [41]:
# calculate how much each user spent
total_rdd = sales_data.reduceByKey(lambda x, y: x + y)
total_rdd.top(10)

[(50, 3663.1519641322166),
 (49, 1640.6599473386934),
 (48, 3163.2191736887394),
 (47, 1339.5497792815631),
 (46, 524.7549005218651),
 (45, 1358.7355048073136),
 (44, 809.4095608362982),
 (43, 4742.08845555165),
 (42, 1252.3463798954997),
 (41, 1932.247614349346)]

In [42]:
# sort the users from highest to lowest spenders
total_rdd.sortBy(lambda x: x[1], ascending=False).collect()

[(22, 5128.6168958368025),
 (4, 5080.275269084312),
 (43, 4742.08845555165),
 (17, 4725.000587104096),
 (3, 3949.9893214752683),
 (50, 3663.1519641322166),
 (39, 3636.468517252868),
 (16, 3611.282851705735),
 (32, 3489.802960208879),
 (5, 3317.9799439591307),
 (48, 3163.2191736887394),
 (23, 3137.455687105382),
 (14, 2904.5867877239316),
 (1, 2861.5932491204994),
 (15, 2831.3507778209305),
 (34, 2813.8167007091097),
 (29, 2787.607758878883),
 (28, 2773.7215831182357),
 (24, 2770.4293186852074),
 (18, 2714.555472416552),
 (20, 2678.284152636498),
 (12, 2652.3793472572015),
 (25, 2606.7528282377975),
 (19, 2374.7930764876705),
 (27, 2374.264042147157),
 (35, 2321.7719178702473),
 (13, 2129.2928036631492),
 (38, 2125.6161779634886),
 (10, 2103.9563568497656),
 (2, 2103.500863802698),
 (30, 2050.8155316282855),
 (41, 1932.247614349346),
 (9, 1819.0792402449536),
 (26, 1724.3303792957267),
 (6, 1682.7737493605562),
 (49, 1640.6599473386934),
 (11, 1613.1624302690284),
 (33, 1459.57640176379

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [44]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda key: key[1], reverse=True)

[(1, 13),
 (23, 12),
 (39, 9),
 (5, 9),
 (11, 9),
 (22, 9),
 (34, 9),
 (50, 8),
 (7, 8),
 (38, 8),
 (18, 8),
 (31, 8),
 (35, 8),
 (30, 7),
 (43, 7),
 (33, 7),
 (13, 7),
 (20, 6),
 (12, 6),
 (17, 5),
 (32, 5),
 (3, 5),
 (19, 5),
 (42, 5),
 (37, 5),
 (40, 5),
 (41, 5),
 (10, 4),
 (16, 4),
 (48, 4),
 (4, 4),
 (2, 4),
 (27, 4),
 (21, 4),
 (29, 4),
 (44, 4),
 (24, 4),
 (26, 3),
 (15, 3),
 (36, 3),
 (45, 3),
 (14, 2),
 (28, 2),
 (49, 2),
 (25, 2),
 (46, 2),
 (9, 1),
 (47, 1),
 (6, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
